# **Data Generation for the League of Legends Championship DB**

### **Importing necessary libraries**

In [67]:
import sqlite3
from faker import Faker
import random
import string
from datetime import datetime, timedelta, date
import requests

In [2]:
fake = Faker()

In [73]:
conn = sqlite3.connect('LOLchamp.db')
cursor = conn.cursor()

### **Generating data for the 'Teams' table**

In [41]:
def generate_random_word(length):
    vowels = 'aeiou'
    consonants = ''.join(set(string.ascii_lowercase) - set(vowels))
    word = ''
    for i in range(length):
        if i % 2 == 0:
            word += random.choice(consonants)
        else:
            word += random.choice(vowels)
    return word.capitalize()

def generate_random_team_name():
    structures = [
        lambda: f"{generate_random_word(random.randint(5, 8))} {generate_random_word(random.randint(5, 8))}",
        lambda: f"{generate_random_word(random.randint(5, 8))} {random.choice(['United', 'FC', 'Gaming', 'Esports'])}",
        lambda: f"Team {generate_random_word(random.randint(5, 8))}",
        lambda: f"{generate_random_word(random.randint(3, 5))}{generate_random_word(random.randint(3, 5))}",
    ]
    return random.choice(structures)()

teams = set()
while len(teams) < 500:
    teams.add(generate_random_team_name())

teams_list = list(teams)
regions = ['NA', 'EU', 'KR', 'CN', 'SEA', 'WS']

for i, team_name in enumerate(teams_list):
    team_id = i + 1
    region = random.choice(regions)
    formation_year = random.randint(2010, 2022)
    total_championships = random.randint(0, 5)

    cursor.execute('''
    INSERT INTO Teams (TeamID, TeamName, Region, FormationYear, TotalChampionships)
    VALUES (?, ?, ?, ?, ?)
    ''', (team_id, team_name, region, formation_year, total_championships))


conn.commit()
conn.close()

### **Generating data for the 'ChampionStats' table**

In [46]:
def champ_names():
    versions_url = 'https://ddragon.leagueoflegends.com/api/versions.json'
    versions = requests.get(versions_url).json()
    latest_version = versions[0]
    champions_url = f'https://ddragon.leagueoflegends.com/cdn/{latest_version}/data/en_US/champion.json'
    champions_data = requests.get(champions_url).json()
    champions_names = list(champions_data['data'].keys())
    return champions_names

all_champions = champ_names()

for i, name in enumerate(all_champions):
    total_picks = random.randint(0, 1000)
    total_bans = random.randint(0, 500)
    win_rate = round(random.uniform(0.4, 0.6), 2)
    cursor.execute(
    '''
    INSERT INTO ChampionStats
    (
    ChampionID,
    ChampionName,
    TotalPicks,
    TotalBans,
    Winrate
    )
    VALUES (?, ?, ?, ?, ?)
    ''', (i + 1, name, total_picks, total_bans, win_rate))

conn.commit()
conn.close()

### **Generating data for the 'Players' table**

In [48]:
cursor.execute("SELECT TeamID FROM Teams")
team_ids = [row[0] for row in cursor.fetchall()]
roles = ['Top', 'Jungle', 'Mid', 'ADC', 'Support']
players_data = []
for i in range(2500):  
    player_id = i + 1
    team_id = random.choice(team_ids)
    nickname = fake.user_name()
    real_name = fake.name()
    role = roles[i % 5]  
    nationality = fake.country()
    join_date = fake.date_between(start_date='-5y', end_date='today')
    players_data.append((player_id, team_id, nickname, real_name, role, nationality, join_date))

cursor.executemany('''
INSERT INTO Players (PlayerID, TeamID, Nickname, RealName, Role, Nationality, JoinDate)
VALUES (?, ?, ?, ?, ?, ?, ?)
''', players_data)

conn.commit()
conn.close()

### **Generating data for the 'Tournaments' table**

In [58]:
tournament_names = [
    "World Championship", "Mid-Season Invitational", "LCK Spring Split", "LCK Summer Split",
    "LPL Spring Split", "LPL Summer Split", "LEC Spring Split", "LEC Summer Split",
    "LCS Spring Split", "LCS Summer Split", "All-Star Event", "Rift Rivals"
]

tournaments_data = []
for i in range(50):  
    tournament_id = i + 1
    tournament_name = f"{random.choice(tournament_names)} {random.randint(2020, 2025)}"
    start_date = fake.date_between(start_date='-5y', end_date='today')
    end_date = start_date + timedelta(days=random.randint(7, 30))  
    base_amount = random.randint(50000, 2000000)  
    if base_amount < 1000000:
        prize_pool = round(base_amount / 1000) * 1000  
    else:
        prize_pool = round(base_amount / 1000000) * 1000000  
        
    tournaments_data.append((tournament_id, tournament_name, start_date, end_date, prize_pool))

cursor.executemany('''
INSERT INTO Tournaments (TournamentID, TournamentName, StartDate, EndDate, PrizePools)
VALUES (?, ?, ?, ?, ?)
''', tournaments_data)

conn.commit()
conn.close()

## **Generating data for the 'Matches' table**

In [68]:
cursor.execute("SELECT TournamentID FROM Tournaments")
tournament_ids = [row[0] for row in cursor.fetchall()]
cursor.execute("SELECT TeamName FROM Teams")
team_names = [row[0] for row in cursor.fetchall()]
matches_data = []
for i in range(1000):  
    match_id = i + 1
    tournament_id = random.choice(tournament_ids)
    team_a, team_b = random.sample(team_names, 2)
    end_date = date(2025, 3, 10)
    match_date = fake.date_between(start_date='-1y', end_date=end_date)
    winning_team = random.choice([team_a, team_b])
    match_duration = random.randint(1200, 3600)
    total_kills = random.randint(10, 50)
    matches_data.append((match_id, tournament_id, team_a, team_b, match_date, winning_team, match_duration, total_kills))
    
cursor.executemany('''
INSERT INTO Matches (MatchID, TournamentID, TeamA, TeamB, MatchDate, WinningTeam, MatchDuration, TotalKills)
VALUES (?, ?, ?, ?, ?, ?, ?, ?)
''', matches_data)

conn.commit()
conn.close()

### **Generating data for the 'ObjectiveControl' table**

In [74]:
cursor.execute("SELECT MatchID, TeamA, TeamB FROM Matches")
matches = cursor.fetchall()
cursor.execute("SELECT TeamName FROM Teams")
team_names = [row[0] for row in cursor.fetchall()]
objective_data = []
for match in matches:
    match_id, team_a_name, team_b_name = match
    first_blood_team = random.choice([team_a_name, team_b_name]) 
    first_baron_team = random.choice([team_a_name, team_b_name]) 
    dragon_kills_a = random.randint(0, 5)
    dragon_kills_b = random.randint(0, 5)
    baron_kills_a = random.randint(0, 3)
    baron_kills_b = random.randint(0, 3)

    objective_data.append((
        match_id,
        team_a_name,
        'Yes' if first_blood_team == team_a_name else 'No',  
        'Yes' if first_baron_team == team_a_name else 'No',  
        dragon_kills_a,
        baron_kills_a
    ))

    objective_data.append((
        match_id,
        team_b_name,
        'Yes' if first_blood_team == team_b_name else 'No', 
        'Yes' if first_baron_team == team_b_name else 'No', 
        dragon_kills_b,
        baron_kills_b
    ))

duplicates_count = int(len(objective_data) * 0.05)
objective_data.extend(random.choices(objective_data, k=duplicates_count))
random.shuffle(objective_data)
for row in objective_data:
    try:
        cursor.execute('''
        INSERT INTO ObjectiveControl (MatchID, TeamName, FirstBlood, FirstBaron, DragonKills, BaronKills)
        VALUES (?, ?, ?, ?, ?, ?)
        ''', row)
    except sqlite3.IntegrityError:
        pass  
        
conn.commit()
conn.close()